---
toc: true
title: 3 Useful Python Code Patterns
---

Over the years I have found out how important it is to constantly keep challenging yourself. Coding can be seen as craftsmanship. A good craftsman will always try to improve the toolbox that they use everyday to perform their work. Code Patterns belong to any good toolbox and when used wisely allow anyone to clearly communicate the intent of their program to both humans and machines. Code is not only something that is executed by a machine but it is also something that is read, understood and maintained by humans.

Some of the programming patterns out there can serve to make code better for both man and machine. Better in this case means that the program is

- easy to understand,
- easy to extend, and
- easy to fix.

Better code at the same time, from a machine perspective, also is

- fast to run,
- reliably handles errors and exceptions, and
- efficiently uses your computer's available resources.

I want to share with you three powerful programming patterns in Python and similar dynamic object-oriented programming languages that can make a big difference and improve your code. Documents like the [Zen of Python](https://www.python.org/dev/peps/pep-0020/) or the [Style Guide for Python Code](https://www.python.org/dev/peps/pep-0008/) give many useful hints on improving your code's ability to communicate intent. The following code patterns can be seen as my personal favorites and can be found one way or another in many of the coding guidelines out there.

# 1. Flat, not nested

The _Zen of Python_ tells you that

> Flat is better than nested

The idea behind flat code being better than nested code is that both in terms of how deeply woven your program's function call graph is and from from a practical point of view how far indented your code ends up being, emphasising a sparse use of _nestedness_ will always lead to better code.

The idea behind flat code being better than nested code is that layering too many paths of execution in your functions can make them hard to understand. Many paths of execution obscure the _main_ path a program will take 99 % of the time and can distract by displaying the minutiae of error handling or data processing with the same priority as the 2 or 3 lines of code that do most of the productive heavy-lifting in a given piece of code.

Let's take a look at a fictitious example involving a web backend that can update user account data after performing a series of validations and input data verifications. In this example, the last step, updating user account data, can only happen if all the previous steps execute correctly.

In [ ]:
def update_user(user, new_data, auth):
    if request.has_perm(auth, 'update_user'):
        if new_data.validate():
            # This is the last step
            # despite appearing almost
            # at the top of the function
            if user.data.update(new_data):
                return "Success"
            else:
                return "Could not update"
        else:
            return "Could not validate"
    else:
        return "Invalid permissions"

Assuming that `return "Success"` is going to be the most used exit in this function we can immediately see that it is not very easy to read. First of all, if the last step is updating the account, then not seeing it as the last step but right in the middle of the function can be confusing. The amount of indentation decreases readability as well, having to scan left and right with your eyes in order to continue downwards breaks the natural flow of reading code.

An alternative approach would be to return early if any of the assertions fall through. This can be seen in the snippet below. Here we check each individual pre-condition and return if it is not satisfied.

In [ ]:
def update_user(user, new_data, auth):
    if not request.has_perm(auth, 'update_user'):
        return "Invalid permissions"

    if not new_data.validate():
        return "Could not validate"
    
    if not user.data.update(new_data):
        return "Could not update"
        
    return "Success"

It is tremendously useful to express complicated sets of pre-conditions in a flat way as much as possible. They allow the reader to quickly scan up and down and see where the actual processing of information happens.

In our function `update_user()` function, the actual step lies in `if not user.data.update(new_data)` and it can be found without having to scan left and right your eyes. 

# 2. Precise Exceptions

Another thing that took me a while to become aware of is the importance of being specific in error handling. For example, given some code that would read out and process sensor data, we can imagine some piece of functionality like this to exist:

In [ ]:
def process():
    result = receive_sensor_data()
    frobnicated = frobnicate(result)
    return renice(result)

We use the function `process()` a few times and discover at some point that `receive_sensor_data()` can fail with a `SensorError`. Out of convenience the whole block of code is then lifted into a `try...except...` block. Usually, when time is running out it is becomes a low priority task to find out where exactly and how exactly failures in an API call can happen. And often, finding the right exception type is difficult, as you might want to be able to handle several exception types correctly. This is how it would typically look like:

In [ ]:
def process():
    try:
        # Only this can throw SensorError
        result = receive_sensor_data()
        frobnicated = frobnicate(result)
        renice(result)
    except SensorError:
        handle_failure()

Good code is all about readability and maintainability. In the above snippet it becomes too easy to forget 6 months later which of the 3 lines after `try:` can actually raise SensorError. Since we know that only `receive_sensor_data()` is able to raise this exception, we readjust our code and we'll take a look and see what has improved.

In [ ]:
def process():
    try:
        result = receive_sensor_data()
    except SensorError:
        handle_failure()
    else:
        frobnicated = frobnicate(result)
        renice(result)

Above, we push the two other lines of code of which we know that they can't raise a `SensorError` into an `else:` block. The `else:` block is executed when the `try:` block runs through its code without any errors. This makes it very clear that only `receive_sensor_data()` can ever cause this exception to be raised, as now it is the only line of code in the `try:` block.

That way there can never be any confusion when the code hasn't been touched in a while about which line might raise the actual exception.

Talking about taking shortcuts when writing error handling code, one thing that happens often is that error handling code does not catch errors precisely enough because the specified exception class is too high up in the inheritance hierarchy of Python exceptions. A useful primer on the hierarchy of built-in errors in Python can be found [here](https://docs.python.org/3/library/exceptions.html#exception-hierarchy).

The useful thing about error handling code is that it not only handles errors, but that it also serves as documentation for later developers that want to understand the failure modes of a piece of code. This matters if the same failing function is used in another block of code somewhere else. A pattern that can often be observed is the following:

In [ ]:
def process():
    try:
        something_complex()
    # Underspecified exception
    # How can the try: block fail?
    except Exception:
        handle_error()

Correctly implementing error handling is not an easy task. It requires you to go through many debugging sessions and studying the documentation of all the different libraries that you are using. Sometimes it might take you 2 weeks to find out that `receive_sensor_data()` will raise the exception `SensorError` and nothing else. This knowledge can be shared immediately by precisely specifying this exception class in the `except:` block.

In [1]:
def process():
    try:
        something_complex()
    # Now it becomes clear how something_complex
    # can fail
    except SomethingComplexException:
        handle_error()

A bonus to this is that well-specified error handling can prevent bugs and other issues with functions that your code is calling to not be caught properly. Maybe you catch `Exception`, and for some reason the code you are using does not convert integers properly and a `ValueError` will be raised. By specifying the exception handling to only take place for `SomethingComplexException`, you will not accidentally bury this issue and bring your program into an invalid state.

# 3. Use Generators

Generators in Python are incredibly useful. A generator is a piece of suspendable code that will execute until a `yield` statement is called from within. The execution can be continued by calling `next()` on it in Python. Typically a generator is used like this:

In [14]:
def generator():
    for i in range(10):
        yield i * 2
        
g = generator()

The generator will run through its code until it hits the first `yield` statement. As a matter of fact, generators simply are Python functions that have at least one `yield` statement inside them. This will immediately turn them into a generator. And more specifically, this will mean that it cannot return values directly to its caller. If we look at the contents of `g`, we will see what `generator()` returns.

In [15]:
g

<generator object generator at 0x110b659e8>

We can continue the execution of the generator and retrieve the first value that it `yield`s by using `next()` on it.

In [16]:
next(g)

0

As we can see, this is nothing more than `i * 2` for `i = 0`. We can call it one more time and retrieve the next value. 

In [17]:
next(g)

2

We can take a shortcut here and retrieve all items at once quite easily:

In [18]:
l = list(generator())
l

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

`list()` in this case, once it receives a generator as an argument, will simply run the generator by repeatedly calling `next()` on it, until it hits a full stop. After that, all the individual items that were retrieved are inserted into a list.

This can be done with `tuple()` as well. In the case of `tuple()`, the resulting object will be a `tuple`, not a `list`:

In [19]:
t = tuple(generator())
t

(0, 2, 4, 6, 8, 10, 12, 14, 16, 18)

We can immediately see that generators provide a lot of flexibility. As a more complicated example, let's consider a function `get_plots()` that goes through a list of items and generates and returns a list of plots to the caller.

In [ ]:
def get_plots(items):
    result = []
    for item in items:
        process_foobar(item)
        reintensify_verticals(item)
        plot = confusion_matrix(item)
        result.append(plot)

In order to use `get_plots()`, one would call it as follows

```python
>>> plots = get_plots(items)
>>> plots
[<plot_1>, <plot_2>]
>>> draw_plots(plots)
...
```

If we think back to the previous generator example, we can take a productive shortcut when defining `get_plots`:

In [ ]:
def get_plots(items):
    for item in items:
        process_foobar(item)
        reintensify_verticals(item)
        yield confusion_matrix(item)

Instead of generating a list and returning it, we leave it to the caller to process the individual plot items returned by `get_plots()`. This allows us to choose how we want the items to be stored in memory by ourselves.

```
>>> plots = tuple(get_plots(items))
>>> plots = list(get_plots_items)
>>> cache.store_list(plots)
>>> # Etc.
```

We can also skip storing the results and instead process them further. For example, if we want to draw the plots we can imagine creating a `draw_plots()` function like so:

In [ ]:
def draw_plots(items):
    for plot in items:
        window.display(plot)

The `for` loop can operate easily on generators and as an example we could display the plots like so:

```
>>> plots = get_plots(items)
>>> plots
<generator object get_plots at 0x...>
>>> draw_plots(plots)
...
```